In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn
import tushare as ts
%matplotlib inline

data = ts.get_k_data('510050','2005-02-23','2018-11-09')
data['d_return'] = data['close'].pct_change()

#读取从股票交易平台导出的数据
data = pd.read_csv('data/510050.txt', encoding='gbk', sep='\t', parse_dates=True, header=1, index_col=0, usecols=[0,1,2,3,4,5], engine='python', skipfooter=2, skipinitialspace=True)

#历史模拟法计算VaR
# np.percentile自动排序然后取n%分位数，注意不能有空值
VaR_5 = np.percentile(data.d_return.dropna(), 5) #一天95%VaR
VaR_1 = np.percentile(data.d_return.dropna(), 1) #一天99%VaR
VaR_1_10 = VaR_1*np.sqrt(10) #十天99%VaR(十天的损失不会超过计算结果)

# 字符串格式化1；
'50ETF一日95%的VaR为 {:.2f}%；99%的VaR为 {:.2f}%'.format(VaR_5*100,VaR_1*100) 

# 字符串格式化2；用%对%进行转义；
'50ETF一日95%%的VaR为 %0.2f%%, 1%%的VaR为 %0.2f%%' % (VaR_5*100,VaR_1*100)

#字符串格式化3
f'50ETF一日5%_VaR为 {VaR_5*100 : .2f}%, 1%_VaR:为 {VaR_1*100 : .2f}%'

plt.figure(figsize= (8,6))
data.d_return.hist(bins=50, alpha=0.6, color='steelblue')
plt.axvline(VaR_5, color='r', linewidth=1,label = 'VaR_5')
plt.axvline(VaR_1, color='y', linewidth=1,label = 'VaR_1')
plt.legend()
plt.xlabel('return')
plt.ylabel('count')

#参数法计算VaR
VaR_1_par=mu_1-2.33*sigma_1
mu_10=mu_1*10
sigma_10=sigma_1*np.sqrt(10)
VaR_10_par=mu_10-2.33*sigma_10

#蒙特卡洛模拟法计算VaR
def single_simulation(s_0, T, n, r, sigma):
    """
    Simulate a single price path. 
    Param n: number of steps. 
    Return: price path as a pandas Series
    """
    p = [s_0, ]
    for i in range(n):
        z = gauss(0,1)
        s_next = p[-1]*exp((r - 0.5*sigma**2)*(T/n) + sigma*sqrt(T/n)*z) #递推公式
        p.append(s_next)
    return p

def GBM(s_0, mu, sigma, T, n):
    delta_t=T/n
    simulated_price=[s_0]
    for i in range(n):
        start_price=simulated_price[i] #获取期初价格
        epsilon=np.random.normal() #按照标准正态分布产生随机数
        end_price=start_price+start_price*(mu*delta_t+sigma*epsilon*np.sqrt(delta_t)) #GBM公式计算期末价格
        end_price=max(0, end_price)
        simulated_price.append(end_price)
    return simulated_price

simulated_price_1=[]
for i in range(10000):
    simulated_price = GBM(s_0, mu_1, sigma_1, 1, 100)
    final_price=simulated_price[-1]
    simulated_prices_1.append(final_price)

simulated_return_1=simulated_prices_1/s_0-1
VaR_1=np.percentile(simulate_return_1, 1)
VaR_10=VaR_1*np.sqrt(10)

#重采样计算VaR
data_pool = data.d_return.dropna()

def sample_VaR(data, size):
    """
    从data中抽取size大小的样本，计算样本的5%和1%VaR
    param data: 产生抽样数据的数据池
    param size: 抽取样本数量
    return: 返回模拟路径下的期末收益
    """
    sample = np.random.choice(data, size, replace=True) #随机抽取size个数据并计算VaR
    VaR_5 = np.percentile(sample, 5)
    VaR_1 = np.percentile(sample, 1)
    return (VaR_5, VaR_1)

# 产生N条收益率路径
samples = [sample_VaR(data_pool, 300) for i in range(1000)]
VaRs = pd.DataFrame(samples, columns=['VaR_5', 'VaR_1'])

VaR_mean = VaRs.mean()

plt.figure(figsize= (8,6))
data.d_return.hist(bins=50, alpha=0.6, color='steelblue')
plt.axvline(VaR_mean.VaR_5, color='r', linewidth=1, label = 'VaR_5')
plt.axvline(VaR_mean.VaR_1, color='y', linewidth=1, label = 'VaR_1')
plt.xlabel('return')
plt.legend()
plt.ylabel('count')